# Counting Values

> Counting unique elements in a sequence in various ways (absolute and cumulative, count and percentage).

In [ ]:
#| default_exp value_counts_plus

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd

In [ ]:
#| export
def value_counts_plus(
    series,
    dropna=False,
    show_top=10,
    sort_others=False,
    style=True,
    size=10,
    thousands=',',
    decimal='.',
    name='data',
    background_gradient='cividis'):
    """
    Provide a few ways of showing counts of values of items in ``series``.

    Parameters
    ----------
    series : pandas Series or list
        A sequence of items to count.
    dropna : bool
        Whether or not to drop missing values.
    show_top : int
        How many of the top rows to display.
    sort_others : bool
        Whether or not to place "Others" in the bottom (default) or in its
        sorted order position.
    style : bool
        Whether or not to style values for easier reading. If set to ``True``
        the result would not be a DataFrame, and cannot be further manipulated.
        Set the value to ``False`` to get aDataFrame as the return value.
    size : int
        The size in points of the font of the table. This results in the whole
        table being resized.
    thousands : str
        The character to use to separate thousands if `style=True`. Defaults to
        `,` but you can change to `.` or space, or any oher character you want.
    decimal : str
        The character to use to display decimal number if `style=True`. Defaults to
        `.` but you can change to `,`or any oher character you want.

    name : str
        The name of the column that you want displayed in the final table. It
        appears in the caption and defaults to "data".
    background_gradient: str
        The name of the color map to be used as the gradient. Many color maps
        are available: cividis, viridis, copper, cool, magma, and more. You can
        reverse the color by appending _r to the end of the colormap name
        cividis_r for example. Enter a random string to get an error message
        with all available colormaps.

    Returns
    -------
    value_counts_df : pandas.DataFrame
        A DataFrame showing counts based on the provided arguments
    """
    final_col_names = ['count', 'cum_count', 'perc', 'cum_perc']
    if name in final_col_names:
        raise ValueError(f"Please make sure you use a name other than {final_col_names}")
    val_counts = pd.Series(series).rename(name).value_counts(dropna=dropna).reset_index()
    val_counts.columns = [name, 'count']
    if len(val_counts) > show_top:
        others_df = pd.DataFrame({
            name: ['Others:'],
            'count': val_counts[show_top:]['count'].sum()
            }, index=[show_top])
        val_counts = pd.concat([
            val_counts[:show_top],
            others_df
        ])
        if sort_others:
            val_counts = val_counts.sort_values(by=['count'], ascending=False)

    count_df = (val_counts
                .assign(
                    cum_count=lambda df: df['count'].cumsum(),
                    perc=lambda df: df['count'].div(df['count'].sum()),
                    cum_perc=lambda df: df['perc'].cumsum())
                )
    if not style:
        return count_df
    count_df.index = range(1, len(count_df)+1)
    count_df.columns = [name, 'count', 'cum. count', '%', 'cum. %']
    return (count_df
            .style
            .format({'count': '{:,}', 'cumsum': '{:,}', 
                     '%': '{:.1%}',
                     'cum. count': '{:,}',
                     'cum. %': '{:.1%}'},
                    thousands=thousands,
                    decimal=decimal)
            .background_gradient(background_gradient)
            .highlight_null()
            .set_caption(f'<h2>Counts of <b>{name}</b></h2>')
            .set_table_attributes(f'style=font-size:{size}pt;'))


In [ ]:
from functools import partial
value_counts_plus = partial(value_counts_plus, size=15)

### Counting a list of status codes - default

In [ ]:
import pandas as pd

import random
import numpy as np
import matplotlib as mpl



In [ ]:
colors = list(mpl.colors.cnames.keys())
colors = random.choices(colors, weights=[0.9, 0.04, 0.05, 0.09]*37, k=10_000)
colors += [np.nan for i in range(240)]
colors[:20]

['aliceblue',
 'lightgreen',
 'mediumaquamarine',
 'mediumpurple',
 'navajowhite',
 'cyan',
 'cornsilk',
 'darkslategray',
 'brown',
 'goldenrod',
 'linen',
 'darkmagenta',
 'darkred',
 'darkorchid',
 'mediumblue',
 'saddlebrown',
 'lightsteelblue',
 'lightgreen',
 'mediumslateblue',
 'cyan']

In [ ]:
#| hide
assert type(value_counts_plus(colors)) == pd.io.formats.style.Styler

### Changing the number of displayed rows with `show_top`

In [ ]:
#| echo
value_counts_plus(colors, show_top=15)

,data,count,cum. count,%,cum. %
1,indigo,264,264,2.6%,2.6%
2,plum,256,520,2.5%,5.1%
3,lavenderblush,248,768,2.4%,7.5%
4,linen,245,"1,013",2.4%,9.9%
5,mediumblue,241,"1,254",2.4%,12.2%
6,nan,240,"1,494",2.3%,14.6%
7,lightsteelblue,240,"1,734",2.3%,16.9%
8,lightgreen,240,"1,974",2.3%,19.3%
9,white,237,"2,211",2.3%,21.6%
10,blanchedalmond,237,"2,448",2.3%,23.9%


In [ ]:
#| hide
assert value_counts_plus(colors, show_top=15, style=False).shape[0] == 16

In [ ]:
#| hide
# make sure it works with low unique values < show_top
assert type(value_counts_plus([1.1, 2, 3, 4], show_top=15)) == pd.io.formats.style.Styler

### Sorting "Others:" 

In [ ]:
#| echo
value_counts_plus(colors, sort_others=True)

,data,count,cum. count,%,cum. %
1,Others:,"7,792","7,792",76.1%,76.1%
2,indigo,264,"8,056",2.6%,78.7%
3,plum,256,"8,312",2.5%,81.2%
4,lavenderblush,248,"8,560",2.4%,83.6%
5,linen,245,"8,805",2.4%,86.0%
6,mediumblue,241,"9,046",2.4%,88.3%
7,nan,240,"9,286",2.3%,90.7%
8,lightsteelblue,240,"9,526",2.3%,93.0%
9,lightgreen,240,"9,766",2.3%,95.4%
10,white,237,"10,003",2.3%,97.7%


In [ ]:
#| hide
assert value_counts_plus(colors, sort_others=True, style=False)['data'].iloc[0] == 'Others:'

## Changing the name of the data and caption title with `name`

In [ ]:
#| echo
value_counts_plus(colors, name='Status codes')

,Status codes,count,cum. count,%,cum. %
1,indigo,264,264,2.6%,2.6%
2,plum,256,520,2.5%,5.1%
3,lavenderblush,248,768,2.4%,7.5%
4,linen,245,"1,013",2.4%,9.9%
5,mediumblue,241,"1,254",2.4%,12.2%
6,nan,240,"1,494",2.3%,14.6%
7,lightsteelblue,240,"1,734",2.3%,16.9%
8,lightgreen,240,"1,974",2.3%,19.3%
9,white,237,"2,211",2.3%,21.6%
10,blanchedalmond,237,"2,448",2.3%,23.9%


## Change the size of table: `size`

In [ ]:
value_counts_plus(colors, size=5)

,data,count,cum. count,%,cum. %
1,indigo,264,264,2.6%,2.6%
2,plum,256,520,2.5%,5.1%
3,lavenderblush,248,768,2.4%,7.5%
4,linen,245,"1,013",2.4%,9.9%
5,mediumblue,241,"1,254",2.4%,12.2%
6,nan,240,"1,494",2.3%,14.6%
7,lightsteelblue,240,"1,734",2.3%,16.9%
8,lightgreen,240,"1,974",2.3%,19.3%
9,white,237,"2,211",2.3%,21.6%
10,blanchedalmond,237,"2,448",2.3%,23.9%


In [ ]:
value_counts_plus(colors, size=20)

,data,count,cum. count,%,cum. %
1,indigo,264,264,2.6%,2.6%
2,plum,256,520,2.5%,5.1%
3,lavenderblush,248,768,2.4%,7.5%
4,linen,245,"1,013",2.4%,9.9%
5,mediumblue,241,"1,254",2.4%,12.2%
6,nan,240,"1,494",2.3%,14.6%
7,lightsteelblue,240,"1,734",2.3%,16.9%
8,lightgreen,240,"1,974",2.3%,19.3%
9,white,237,"2,211",2.3%,21.6%
10,blanchedalmond,237,"2,448",2.3%,23.9%


## Completely change the caption using an HTML string with `set_caption`

In [ ]:
caption = '<h4>Status codes</h4>Top 5 values <a href="https://example.com">raw data</a>'
value_counts_plus(
    colors,
    name='Statuses',
    show_top=5).set_caption(caption)

,Statuses,count,cum. count,%,cum. %
1,indigo,264,264,2.6%,2.6%
2,plum,256,520,2.5%,5.1%
3,lavenderblush,248,768,2.4%,7.5%
4,linen,245,"1,013",2.4%,9.9%
5,mediumblue,241,"1,254",2.4%,12.2%
6,Others:,"8,986","10,240",87.8%,100.0%


### Removing table styling if you want a pure `DataFrame`:
* Counting in non-styled DataFrames is 0-based in case you want to further process it
* Columns are displayed in a slightly different manner

In [ ]:
value_counts_plus(colors, style=False)

,data,count,cum_count,perc,cum_perc
0,indigo,264,264,0.025781,0.025781
1,plum,256,520,0.025000,0.050781
2,lavenderblush,248,768,0.024219,0.075000
3,linen,245,1013,0.023926,0.098926
4,mediumblue,241,1254,0.023535,0.122461
5,NaN,240,1494,0.023438,0.145898
6,lightsteelblue,240,1734,0.023438,0.169336
7,lightgreen,240,1974,0.023438,0.192773
8,white,237,2211,0.023145,0.215918
9,blanchedalmond,237,2448,0.023145,0.239062


In [ ]:
#| hide
assert type(value_counts_plus(colors, style=False)) == pd.DataFrame

### Changing the theme used `theme`

In [ ]:
value_counts_plus(colors, background_gradient='Greens')

,data,count,cum. count,%,cum. %
1,indigo,264,264,2.6%,2.6%
2,plum,256,520,2.5%,5.1%
3,lavenderblush,248,768,2.4%,7.5%
4,linen,245,"1,013",2.4%,9.9%
5,mediumblue,241,"1,254",2.4%,12.2%
6,nan,240,"1,494",2.3%,14.6%
7,lightsteelblue,240,"1,734",2.3%,16.9%
8,lightgreen,240,"1,974",2.3%,19.3%
9,white,237,"2,211",2.3%,21.6%
10,blanchedalmond,237,"2,448",2.3%,23.9%


In [ ]:
value_counts_plus(colors, background_gradient='cool')

,data,count,cum. count,%,cum. %
1,indigo,264,264,2.6%,2.6%
2,plum,256,520,2.5%,5.1%
3,lavenderblush,248,768,2.4%,7.5%
4,linen,245,"1,013",2.4%,9.9%
5,mediumblue,241,"1,254",2.4%,12.2%
6,nan,240,"1,494",2.3%,14.6%
7,lightsteelblue,240,"1,734",2.3%,16.9%
8,lightgreen,240,"1,974",2.3%,19.3%
9,white,237,"2,211",2.3%,21.6%
10,blanchedalmond,237,"2,448",2.3%,23.9%


### Convert the table to raw HTML for embedding in a blog or web page

In [ ]:
print(value_counts_plus(colors).to_html()[:700])

<style type="text/css">
#T_a7084_row0_col1, #T_a7084_row0_col2, #T_a7084_row0_col3, #T_a7084_row0_col4, #T_a7084_row1_col1, #T_a7084_row1_col3, #T_a7084_row2_col1, #T_a7084_row2_col3, #T_a7084_row3_col1, #T_a7084_row3_col3, #T_a7084_row4_col1, #T_a7084_row4_col3, #T_a7084_row5_col1, #T_a7084_row5_col3, #T_a7084_row6_col1, #T_a7084_row6_col3, #T_a7084_row7_col1, #T_a7084_row7_col3, #T_a7084_row8_col1, #T_a7084_row8_col3, #T_a7084_row9_col1, #T_a7084_row9_col3 {
  background-color: #00224e;
  color: #f1f1f1;
}
#T_a7084_row1_col2, #T_a7084_row1_col4 {
  background-color: #002758;
  color: #f1f1f1;
}
#T_a7084_row2_col2, #T_a7084_row2_col4 {
  background-color: #002b62;
  color: #f1f1f1;
}
#T_a70


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()